In [1]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [2]:
import galois
import numpy as np
import time

In [3]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(2**3)

In [4]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [5]:
#ho verificato che funziona pure con i random
#A = field.Random((k, n_k))
#print(A)

#questa è con l'esempio di 4.3
A = field([[1,2,4,7],
           [0,2,3,1],
           [2,3,2,0]])

In [6]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [7]:
A_primo = field(matrice_bin1 @ A @ matrice_bin2)
print(A_primo)

[[2 0 3 2]
 [1 7 2 4]
 [0 1 2 3]]


In [8]:
A_primo = field([[2,2,0,3],
           [1,4,7,2],
           [0,3,1,2]])

In [9]:
#definito funzione ordinamento lessicografico riga (ordino ogni riga e definisco ordine degli indici) e gestiti casi di 
#conflitto. L'argomento colonne è quello che ci indica in che ordine verificare i conflitti 
#ex: colonne=[0,1,2] --> ordino per colonna 0, se c'è conflitto tra due vedo la colonna 1 e cosi via

#faccio ordinamento crescente dei valori all'interno della riga e torno matrice con righe ordinate (ordinato).
#da questa matrice faccio l'ordinamento elemento a elemento posizionale corrispondente e definisco quindi gli indici 
#riordinati. ex: [1 0 2] --> prima la riga ORIGINALE 1 poi riga ORIGINALE 0 e così via. Per originale intendo "matrice"

def ordina_per_riga(matrice, colonne):
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
        
    ordinato = np.array(ordinato)
    
    #inverto ordine colonne [::-1] poiché lexsort parte dall'ultima colonna
    indici_ordinamento = np.lexsort([ordinato[:, col] for col in colonne[::-1]])
    print("\n",indici_ordinamento)
    print("Matrice ordinata per righe: ",matrice[indici_ordinamento])
    return matrice[indici_ordinamento]

#ordino poi per il primo elemento della colonna e in caso di conflitti li ho gestiti. 
def ordina_per_colonna(matrice, righe):
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
    return matrice[:, indici_ordinamento]

#qui faccio il test
def test_uguaglianza(matrice1, matrice2):
    colonne = range(n_k)
    righe = range(k)
    print("Matrice1: ", matrice1)

    ordinata1 = ordina_per_colonna(ordina_per_riga(matrice1, colonne), righe)
    print("\nOrdinata 1: ",ordinata1)

    print("Matrice2: ", matrice2)
    ordinata2 = ordina_per_colonna(ordina_per_riga(matrice2, colonne), righe)
    print("\nOrdinata2: ",ordinata2)     

    if np.array_equal(ordinata1, ordinata2):
        print("Le due matrici hanno la stessa CF.")
        return True
    else:
        print("Le due matrici hanno CF diversa")
        return False   



In [10]:
test_uguaglianza(A, A_primo)

Matrice1:  [[1 2 4 7]
 [0 2 3 1]
 [2 3 2 0]]

 [1 2 0]
Matrice ordinata per righe:  [[0 2 3 1]
 [2 3 2 0]
 [1 2 4 7]]

Ordinata 1:  [[0 1 2 3]
 [2 0 3 2]
 [1 7 2 4]]
Matrice2:  [[2 2 0 3]
 [1 4 7 2]
 [0 3 1 2]]

 [2 0 1]
Matrice ordinata per righe:  [[0 3 1 2]
 [2 2 0 3]
 [1 4 7 2]]

Ordinata2:  [[0 1 2 3]
 [2 0 3 2]
 [1 7 2 4]]
Le due matrici hanno la stessa CF.


True

In [22]:
#FACCIO METODO DI MONTECARLO
#numero iterazioni Monte Carlo
num_iterazioni = 10000  

# Funzione per generare matrici binarie casuali
def genera_matrice_binaria(dimensioni, campo):
    matrice_binaria = np.random.randint(0, 2, size=dimensioni)
    return campo(matrice_binaria)

# Funzione per ordinare le righe con contatori di ordinamenti e swap
def ordina_per_riga_con_contatori(matrice, colonne, num_swap_righe):
    # Mantieni traccia della posizione originale delle righe
    posizioni_originali = np.arange(matrice.shape[0])
    
    # Ordina gli elementi all'interno di ogni riga
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    
    # Trova gli indici di ordinamento usando np.lexsort sulle colonne
    indici_ordinamento = np.lexsort([ordinato[:, col] for col in colonne[::-1]])
    
    # Conta gli swap solo quando la posizione delle righe cambia
    for i, indice in enumerate(indici_ordinamento):
        if indice != posizioni_originali[i]:
            num_swap_righe += 1
    
    # Riordina la matrice in base agli indici trovati
    matrice_ordinata = matrice[indici_ordinamento]
    
    return matrice_ordinata, num_swap_righe/2

# Funzione per ordinare le colonne con contatori di ordinamenti e swap
def ordina_per_colonna_con_contatori(matrice, righe, num_swap_colonne):
    # Mantieni traccia della posizione originale delle colonne
    posizioni_originali = np.arange(matrice.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, indice in enumerate(indici_ordinamento):
        if indice != posizioni_originali[i]:
            num_swap_colonne += 1
    
    # Riordina la matrice in base agli indici trovati
    matrice_ordinata = matrice[:, indici_ordinamento]
    # Faccio num_swap_colonne/2 poichè contando il numero di diversi ne conta ovviamente il doppio. 
    # Ex: (0, 1, 2, 3) potrebbe diventare (0, 1, 3, 2) il contatore sarebbe a due, ma io ho fatto uno swap
    return matrice_ordinata, num_swap_colonne/2

# Funzione test con contatori
def test_uguaglianza_con_contatori(matrice1, matrice2, num_swap_righe, num_swap_colonne):
    start_time = time.perf_counter()
    #intervallo per le colonne
    colonne = range(n_k)  
    #intervallo corretto per le righe
    righe = range(k)      
    # Ordinamento matrice1
    ordinata1, num_swap_righe = ordina_per_riga_con_contatori(matrice1, colonne, num_swap_righe)
    ordinata1, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata1, righe, num_swap_colonne)
    
    # Ordinamento matrice2
    ordinata2, num_swap_righe = ordina_per_riga_con_contatori(matrice2, colonne, num_swap_righe)
    ordinata2, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata2, righe, num_swap_colonne)
    end_time = time.perf_counter()
    durata = (end_time - start_time)*1000
    # Confronto
    return np.array_equal(ordinata1, ordinata2), num_swap_righe, num_swap_colonne, durata

# Funzione per eseguire la simulazione di Monte Carlo
def simulazione_montecarlo(num_iterazioni):
    #contatore simulazioni positive
    simulazioni_positive = 0
    #contatore swap righe
    num_swap_righe = 0  
    #contatore swap colonne
    num_swap_colonne = 0  
    #inizializzo la durata
    tempo_esecuzione = 0.0
    
    for _ in range(num_iterazioni):
    
        A = field(np.random.randint(0, field.order, size=(k, n_k)))
        matrice_bin1 = genera_matrice_binaria((k, k), field)
        matrice_bin2 = genera_matrice_binaria((n_k, n_k), field)

        A_primo = field(matrice_bin1 @ A @ matrice_bin2)

        #faccio il test di uguaglianza tra A e A_primo
        uguaglianza, num_swap_righe, num_swap_colonne, durata = test_uguaglianza_con_contatori(A, A_primo, num_swap_righe, num_swap_colonne)
        tempo_esecuzione += durata
        if uguaglianza:
            simulazioni_positive += 1
    
    # Output statistiche
    print()
    print(f"Tempo esecuzione medio: {tempo_esecuzione/num_iterazioni} ms")
    print(f"Numero medio di swap righe: {num_swap_righe}")
    print(f"Numero medio di swap colonne: {num_swap_colonne}")
    print(f"Numero totale di test positivi: {simulazioni_positive}")

# Esegui la simulazione
simulazione_montecarlo(num_iterazioni)



Tempo esecuzione medio: 0.21942467909639163 ms
Numero medio di swap righe: 1.9664800204545845
Numero medio di swap colonne: 3.865996999571585
Numero totale di test positivi: 0
